<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/src/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and dependencies
Run the below cell, which shall clone the repo contents and prepare the machine for work. This may take a few minutes.

***Important:*** This is the first cell you shall run in this project, otherwise it won't be working.

After 'pip' is done, it may ask you to restart this session - please do accordingly.

In [6]:
# remove possible leftovers and clear-clone the whole repo
%cd /content
%rm -rf ./machine_learning_ev_ice
!git clone https://github.com/stas420/machine_learning_ev_ice.git

# a git pull pro forma
%cd ./machine_learning_ev_ice
!git pull

# go to our main directory
%cd ./src

# check (and download, if needed) dependencies for the project
%pip install -r requirements.txt

/content
Cloning into 'machine_learning_ev_ice'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 32), reused 39 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 5.70 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/machine_learning_ev_ice
Already up to date.
/content/machine_learning_ev_ice/src


# małnt dejta

tu sb poberamy dane z podmontowanego dysku gugl

In [2]:
%cd /content

from google.colab import drive
import os

if not 'drive' in os.listdir():
  drive.mount('/content/drive')
else:
  print("drajw dajrektory alredi prezent")

/content
Mounted at /content/drive


### a tu sb dodamy zmienne globalne UwU

In [3]:
VIDEOS_PATH = '/content/drive/MyDrive/videos'
IMAGES_PATH = '/content/imgs'

# extrakt frejms

In [4]:
# clearing leftovers pro forma
%rm -rf /content/imgs
%mkdir /content/imgs

import cv2
import os

# function which takes a .mp4 file and extracts its frames,
# then saves them as images in the provided directory
def extract_frames(video_path, output_dir, fps=1) -> None:
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    saved = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        else:
            frame_path = os.path.join(output_dir, f"frame_{saved:03d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1
    cap.release()

# image extraction
for x in os.listdir(VIDEOS_PATH):
  extract_frames(VIDEOS_PATH + '/' + x, "/content/imgs/" + x, fps=1)

!ls -la ./content/imgs

KeyboardInterrupt: 

# is grin plejt

In [9]:
%cd /content/machine_learning_ev_ice/src

# for development only, suck my ass!
%rm -rf /content/results
from ultralytics import YOLO
import cv2
import os

def is_green_plate(plate_img):
    hsv = cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV)
    h, s, _ = hsv.mean(axis=0).mean(axis=0)
    return 35 < h < 85 and s > 40

model = YOLO('./yolov8n.pt')
input_dir = "/content/imgs/2024-01-29.mp4"
output_dir = "/content/results"
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(input_dir):
    if not fname.endswith(".jpg"):
        continue
    path = os.path.join(input_dir, fname)
    img = cv2.imread(path)
    results = model(img)

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = img[y1:y2, x1:x2]
        label = "green" if is_green_plate(crop) else "normal"
        color = (0,255,0) if label == "green" else (255,0,0)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imwrite(os.path.join(output_dir, fname), img)


/content/machine_learning_ev_ice/src

0: 384x640 (no detections), 165.8ms
Speed: 4.6ms preprocess, 165.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 150.5ms
Speed: 5.1ms preprocess, 150.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 150.6ms
Speed: 5.6ms preprocess, 150.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 148.5ms
Speed: 5.1ms preprocess, 148.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 150.3ms
Speed: 5.3ms preprocess, 150.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20 cars, 1 truck, 1 traffic light, 154.5ms
Speed: 5.0ms preprocess, 154.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 166.4ms
Speed: 4.6ms preprocess, 166.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21 cars, 148.6

# prawdziwe mlowanie


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# 2. Wczytanie modelu YOLO (np. wytrenowany na tablicach)
model = YOLO("../models/license_plate.pt")

# 3. Wczytanie przykładowego obrazu
image_path = "../data/frames/frame_01.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 4. Detekcja tablic rejestracyjnych
results = model(image)

# 5. Analiza i klasyfikacja kolorów tablic
for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped = image[y1:y2, x1:x2]

    # Analiza koloru – przejście na HSV
    hsv = cv2.cvtColor(cropped, cv2.COLOR_BGR2HSV)
    avg_color = hsv.mean(axis=0).mean(axis=0)

    # Detekcja zielonej barwy
    h, s, v = avg_color
    is_green = 35 < h < 85 and s > 40

    color_label = "ZIELONA (elektryczny)" if is_green else "Zwykła"

    # Rysowanie wyników
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0) if is_green else (255,0,0), 2)
    cv2.putText(image, color_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                (0,255,0) if is_green else (255,0,0), 2)

# 6. Wyświetlenie wyników
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


tensor([[0.9320, 0.5988, 0.1856],
        [0.5191, 0.3469, 0.9685],
        [0.1125, 0.9076, 0.0556],
        [0.0212, 0.8824, 0.7880],
        [0.8299, 0.5203, 0.5885]])


Dżem dobry
:)